In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.colors as col
import seaborn as sns
import ast

# Generic Functions

In [2]:
def get_training_hyper(row, hyper="seed"):
    hypers = ast.literal_eval(row["config/training"])
    return hypers[hyper]

def get_problem_hyper(row, hyper="lam"):
    hypers = ast.literal_eval(row["config/problem"])
    return hypers[hyper]

def get_d_hypers(row, hyper="n_hidden_units"):
    hypers = ast.literal_eval(row["config/discriminator"])
    return hypers[hyper] 

def get_g_hypers(row, hyper="n_hidden_units"):
    hypers = ast.literal_eval(row["config/generator"])
    return hypers[hyper] 

def get_expanded_df(results_df, optim="sgd", lam=False):
    
    # List the training hyperparameters
    if optim == "sgd":
        training_hypers = ["seed", "d_lr", "g_lr", "gamma", "momentum", "step_size"]
        precise_cols = training_hypers[1:5]
    elif optim == "adam":
        training_hypers = ["seed", "d_lr", "g_lr", "gamma", "g_betas", "d_betas", "step_size"]          
        precise_cols = training_hypers[1:4]
    else:
        raise Exception(f"optimizer {optim} not recognized")

    # List the discriminator and generator network hyperparameters
    nn_hypers = ["n_hidden_units", "n_hidden_layers"]

    # Add a new column for each training hyperparameter
    for hyper in training_hypers:
        results_df[hyper] = results_df.apply(lambda row: get_training_hyper(row, hyper), axis=1)

    # Add a new column for each discriminator and generator hyperparameter
    for hyper in nn_hypers:
        results_df["d_"+hyper] = results_df.apply(lambda row: get_d_hypers(row, hyper), axis=1)
        results_df["g_"+hyper] = results_df.apply(lambda row: get_g_hypers(row, hyper), axis=1)
        
    # Add a column for lambda
    if lam:
        precise_cols.append("lam")
        results_df["lam"] = results_df.apply(lambda row: get_problem_hyper(row, "lam"), axis=1)
        
    # Increase precision of several hypers
    for col in precise_cols:
        results_df[col] = results_df[col].map(lambda x: '{0:.17}'.format(x))
        
    # Drop unnecessary columns
    results_df = results_df.drop(["time_this_iter_s", "done", "timesteps_total", "episodes_total", 
                                  "training_iteration", "experiment_id", "timestamp", "time_total_s", 
                                  "pid", "node_ip", "time_since_restore", "timesteps_since_restore", 
                                  "iterations_since_restore", "trial_id", "config/generator", 
                                  "config/discriminator", "config/problem", "config/training"], axis=1)   
    return results_df

# 1. EXP

## 1.1 L2

In [10]:
# Read in the EXP results
exp_results = pd.read_csv("classical_results/ray_tune_exp_MSELoss.csv", usecols=np.arange(1,22))
exp_results = get_expanded_df(exp_results, optim="adam")
exp_results.loc[exp_results["mean_squared_error"].idxmin()]

mean_squared_error                                  4.03879e-08
date                                        2022-03-15_00-36-33
hostname                         holy7c24202.rc.fas.harvard.edu
seed                                                          3
d_lr                                       0.003998154054910664
g_lr                                       0.011280590164998256
gamma                                       0.92827969684332612
g_betas               [0.18188980961316425, 0.9952247116684417]
d_betas               [0.22651674714417291, 0.7427481558494436]
step_size                                                    10
d_n_hidden_units                                             20
g_n_hidden_units                                             20
d_n_hidden_layers                                             3
g_n_hidden_layers                                             4
Name: 49, dtype: object

## 1.2 L1

In [11]:
# Read in the EXP results
exp_results = pd.read_csv("classical_results/ray_tune_exp_L1Loss.csv", usecols=np.arange(1,22))
exp_results = get_expanded_df(exp_results, optim="adam")
exp_results.loc[exp_results["mean_squared_error"].idxmin()]

mean_squared_error                                   0.00013617
date                                        2022-03-15_00-44-38
hostname                         holy7c24202.rc.fas.harvard.edu
seed                                                          8
d_lr                                       0.019631921422541127
g_lr                                      0.0067638346679588651
gamma                                       0.90847825545056637
g_betas               [0.21733559890544668, 0.6981208782506974]
d_betas               [0.22916606500920175, 0.9782037648205828]
step_size                                                     9
d_n_hidden_units                                             20
g_n_hidden_units                                             30
d_n_hidden_layers                                             2
g_n_hidden_layers                                             5
Name: 90, dtype: object

## 1.3 Huber

In [12]:
# Read in the EXP results
exp_results = pd.read_csv("classical_results/ray_tune_exp_SmoothL1Loss.csv", usecols=np.arange(1,22))
exp_results = get_expanded_df(exp_results, optim="adam")
exp_results.loc[exp_results["mean_squared_error"].idxmin()]

mean_squared_error                                  2.85947e-08
date                                        2022-03-15_00-54-03
hostname                         holy7c24202.rc.fas.harvard.edu
seed                                                          4
d_lr                                       0.012751363190218971
g_lr                                       0.011853007884969579
gamma                                       0.96223535966670204
g_betas               [0.20733256235866268, 0.8834288835546434]
d_betas                [0.19180760980017156, 0.873570627966314]
step_size                                                     4
d_n_hidden_units                                             20
g_n_hidden_units                                             20
d_n_hidden_layers                                             2
g_n_hidden_layers                                             4
Name: 143, dtype: object

# 2. SHO

## 2.1 L2

In [21]:
# Read in the first SHO results
sho_results = pd.read_csv("classical_results/ray_tune_sho_MSELoss.csv", usecols=np.arange(1,22))
sho_results = get_expanded_df(sho_results, optim="adam")
sho_results.loc[sho_results["mean_squared_error"].idxmin()]

mean_squared_error                                   1.36245e-09
date                                         2022-03-15_14-13-56
hostname                          holy7c24304.rc.fas.harvard.edu
seed                                                           1
d_lr                                       0.0064142351844139762
g_lr                                        0.010700740751825946
gamma                                         0.9966532790839987
g_betas               [0.056260393978402594, 0.9856354179096971]
d_betas                 [0.27653985619978705, 0.824622770493504]
step_size                                                      4
d_n_hidden_units                                              50
g_n_hidden_units                                              20
d_n_hidden_layers                                              4
g_n_hidden_layers                                              2
Name: 29, dtype: object

## 2.2 L1

In [22]:
# Read in the first SHO results
sho_results = pd.read_csv("classical_results/ray_tune_sho_L1Loss.csv", usecols=np.arange(1,22))
sho_results = get_expanded_df(sho_results, optim="adam")
sho_results.loc[sho_results["mean_squared_error"].idxmin()]

mean_squared_error                                   1.40999e-05
date                                         2022-03-15_14-32-44
hostname                          holy7c24304.rc.fas.harvard.edu
seed                                                           8
d_lr                                        0.017644664663183745
g_lr                                        0.017716116013755272
gamma                                        0.96579292102052083
g_betas                 [0.3255239388319677, 0.7932651019648391]
d_betas               [0.003087774244473942, 0.7256657996354898]
step_size                                                     17
d_n_hidden_units                                              50
g_n_hidden_units                                              40
d_n_hidden_layers                                              2
g_n_hidden_layers                                              5
Name: 186, dtype: object

## 2.3 Huber

In [23]:
# Read in the first SHO results
sho_results = pd.read_csv("classical_results/ray_tune_sho_SmoothL1Loss.csv", usecols=np.arange(1,22))
sho_results = get_expanded_df(sho_results, optim="adam")
sho_results.loc[sho_results["mean_squared_error"].idxmin()]

mean_squared_error                                  5.29674e-10
date                                        2022-03-15_14-40-39
hostname                         holy7c24304.rc.fas.harvard.edu
seed                                                          4
d_lr                                       0.010844361884840359
g_lr                                      0.0069910126017160689
gamma                                       0.98584169381562026
g_betas               [0.08826700490681576, 0.8232697033306144]
d_betas                [0.2244857669030433, 0.8492048870694309]
step_size                                                    20
d_n_hidden_units                                             20
g_n_hidden_units                                             20
d_n_hidden_layers                                             2
g_n_hidden_layers                                             2
Name: 29, dtype: object

# 3. NLO

## 3.1 L2 

In [24]:
# Read in the first NLO results
nlo_results = pd.read_csv("classical_results/ray_tune_nlo_MSELoss.csv", usecols=np.arange(1,22))
nlo_results = get_expanded_df(nlo_results, optim="adam")
nlo_results.loc[nlo_results["mean_squared_error"].idxmin()]

mean_squared_error                                  3.82057e-10
date                                        2022-03-15_14-20-38
hostname                         holy7c24401.rc.fas.harvard.edu
seed                                                          3
d_lr                                       0.012452364544839624
g_lr                                      0.0072900784938496324
gamma                                        0.9925769778492004
g_betas               [0.14273764411127707, 0.7346959799270164]
d_betas               [0.23073355192717207, 0.9037974958572559]
step_size                                                     7
d_n_hidden_units                                             40
g_n_hidden_units                                             50
d_n_hidden_layers                                             3
g_n_hidden_layers                                             3
Name: 170, dtype: object

## 3.2 L1

In [25]:
# Read in the first NLO results
nlo_results = pd.read_csv("classical_results/ray_tune_nlo_L1Loss.csv", usecols=np.arange(1,22))
nlo_results = get_expanded_df(nlo_results, optim="adam")
nlo_results.loc[nlo_results["mean_squared_error"].idxmin()]

mean_squared_error                                 0.000137843
date                                       2022-03-15_14-30-57
hostname                        holy7c24401.rc.fas.harvard.edu
seed                                                         2
d_lr                                    0.00050559757595072818
g_lr                                      0.016135911992710888
gamma                                      0.99123626313892488
g_betas               [0.2019346402168863, 0.9445348566539493]
d_betas                [0.07560024869342562, 0.70241759885858]
step_size                                                   20
d_n_hidden_units                                            20
g_n_hidden_units                                            40
d_n_hidden_layers                                            2
g_n_hidden_layers                                            3
Name: 3, dtype: object

## 3.3 Huber

In [26]:
# Read in the first NLO results
nlo_results = pd.read_csv("classical_results/ray_tune_nlo_SmoothL1Loss.csv", usecols=np.arange(1,22))
nlo_results = get_expanded_df(nlo_results, optim="adam")
nlo_results.loc[nlo_results["mean_squared_error"].idxmin()]

mean_squared_error                                  1.45709e-10
date                                        2022-03-15_14-46-58
hostname                         holy7c24401.rc.fas.harvard.edu
seed                                                          6
d_lr                                      0.0031221646163788852
g_lr                                      0.0017296631326595539
gamma                                       0.98498488568926723
g_betas               [0.08783000910332427, 0.9926995801150367]
d_betas               [0.10201458685483214, 0.7911442537985977]
step_size                                                     6
d_n_hidden_units                                             40
g_n_hidden_units                                             50
d_n_hidden_layers                                             3
g_n_hidden_layers                                             5
Name: 124, dtype: object

# 4. POS

## 4.1 L2

In [28]:
# Read in the first POS results
pos_results = pd.read_csv("classical_results/ray_tune_pos_MSELoss.csv", usecols=np.arange(1,22))
pos_results = get_expanded_df(pos_results, optim="adam")
pos_results.loc[pos_results["mean_squared_error"].idxmin()]

mean_squared_error                                   1.4003e-10
date                                        2022-03-15_14-54-29
hostname                         holy7c24304.rc.fas.harvard.edu
seed                                                          4
d_lr                                       0.017959771526245775
g_lr                                       0.011193423968088733
gamma                                       0.94271382528618619
g_betas               [0.05908725061529524, 0.8673581223588395]
d_betas               [0.20984586474745626, 0.9287130645373443]
step_size                                                    12
d_n_hidden_units                                             20
g_n_hidden_units                                             30
d_n_hidden_layers                                             4
g_n_hidden_layers                                             2
Name: 97, dtype: object

## 4.2 L1

In [29]:
# Read in the first POS results
pos_results = pd.read_csv("classical_results/ray_tune_pos_L1Loss.csv", usecols=np.arange(1,22))
pos_results = get_expanded_df(pos_results, optim="adam")
pos_results.loc[pos_results["mean_squared_error"].idxmin()]

mean_squared_error                                  9.39328e-06
date                                        2022-03-15_15-04-45
hostname                         holy7c24304.rc.fas.harvard.edu
seed                                                          9
d_lr                                      0.0072471212701768067
g_lr                                       0.007817139511342594
gamma                                       0.93213533835205242
g_betas                [0.2696403474486619, 0.9860848390188193]
d_betas               [0.07327190318329017, 0.8357632100613337]
step_size                                                     8
d_n_hidden_units                                             20
g_n_hidden_units                                             50
d_n_hidden_layers                                             4
g_n_hidden_layers                                             4
Name: 114, dtype: object

## 4.3 Huber

In [30]:
# Read in the first POS results
pos_results = pd.read_csv("classical_results/ray_tune_pos_SmoothL1Loss.csv", usecols=np.arange(1,22))
pos_results = get_expanded_df(pos_results, optim="adam")
pos_results.loc[pos_results["mean_squared_error"].idxmin()]

mean_squared_error                                  1.51619e-10
date                                        2022-03-15_15-15-11
hostname                         holy7c24304.rc.fas.harvard.edu
seed                                                          3
d_lr                                      0.0068643724558994819
g_lr                                      0.0055824955430016808
gamma                                       0.90881459598615855
g_betas                 [0.3006113383779244, 0.867007901138201]
d_betas               [0.17466932684331452, 0.6755278202266661]
step_size                                                    16
d_n_hidden_units                                             50
g_n_hidden_units                                             30
d_n_hidden_layers                                             3
g_n_hidden_layers                                             2
Name: 132, dtype: object

# 5. SIR

## 5.1 L2

In [31]:
# Read in the SIR results
sir_results = pd.read_csv("classical_results/ray_tune_pos_MSELoss.csv", usecols=np.arange(1,22))
sir_results = get_expanded_df(sir_results, optim="adam")
sir_results.loc[sir_results["mean_squared_error"].idxmin()]

mean_squared_error                                   1.4003e-10
date                                        2022-03-15_14-54-29
hostname                         holy7c24304.rc.fas.harvard.edu
seed                                                          4
d_lr                                       0.017959771526245775
g_lr                                       0.011193423968088733
gamma                                       0.94271382528618619
g_betas               [0.05908725061529524, 0.8673581223588395]
d_betas               [0.20984586474745626, 0.9287130645373443]
step_size                                                    12
d_n_hidden_units                                             20
g_n_hidden_units                                             30
d_n_hidden_layers                                             4
g_n_hidden_layers                                             2
Name: 97, dtype: object

## 5.2 L1

In [32]:
# Read in the SIR results
sir_results = pd.read_csv("classical_results/ray_tune_pos_L1Loss.csv", usecols=np.arange(1,22))
sir_results = get_expanded_df(sir_results, optim="adam")
sir_results.loc[sir_results["mean_squared_error"].idxmin()]

mean_squared_error                                  9.39328e-06
date                                        2022-03-15_15-04-45
hostname                         holy7c24304.rc.fas.harvard.edu
seed                                                          9
d_lr                                      0.0072471212701768067
g_lr                                       0.007817139511342594
gamma                                       0.93213533835205242
g_betas                [0.2696403474486619, 0.9860848390188193]
d_betas               [0.07327190318329017, 0.8357632100613337]
step_size                                                     8
d_n_hidden_units                                             20
g_n_hidden_units                                             50
d_n_hidden_layers                                             4
g_n_hidden_layers                                             4
Name: 114, dtype: object

## 5.3 Huber

In [33]:
# Read in the SIR results
sir_results = pd.read_csv("classical_results/ray_tune_pos_SmoothL1Loss.csv", usecols=np.arange(1,22))
sir_results = get_expanded_df(sir_results, optim="adam")
sir_results.loc[sir_results["mean_squared_error"].idxmin()]

mean_squared_error                                  1.51619e-10
date                                        2022-03-15_15-15-11
hostname                         holy7c24304.rc.fas.harvard.edu
seed                                                          3
d_lr                                      0.0068643724558994819
g_lr                                      0.0055824955430016808
gamma                                       0.90881459598615855
g_betas                 [0.3006113383779244, 0.867007901138201]
d_betas               [0.17466932684331452, 0.6755278202266661]
step_size                                                    16
d_n_hidden_units                                             50
g_n_hidden_units                                             30
d_n_hidden_layers                                             3
g_n_hidden_layers                                             2
Name: 132, dtype: object

# 6. COO

## 6.1 L2

In [34]:
# Read in the COO results
coo_results = pd.read_csv("classical_results/ray_tune_coo_MSELoss.csv", usecols=np.arange(1,22))
coo_results = get_expanded_df(coo_results, optim="adam")
coo_results.loc[coo_results["mean_squared_error"].idxmin()]

mean_squared_error                                  2.84311e-07
date                                        2022-03-15_16-25-27
hostname                         holy7c24401.rc.fas.harvard.edu
seed                                                          3
d_lr                                       0.016821084233699227
g_lr                                     0.00034120745965572722
gamma                                       0.99766896035885932
g_betas               [0.22395595814116892, 0.9872786232130291]
d_betas               [0.16171068799799587, 0.8545270451606666]
step_size                                                    17
d_n_hidden_units                                             40
g_n_hidden_units                                             50
d_n_hidden_layers                                             2
g_n_hidden_layers                                             3
Name: 178, dtype: object

In [35]:
# Read in the COO results
coo_results = pd.read_csv("classical_results/ray_tune_coo_L1Loss.csv", usecols=np.arange(1,22))
coo_results = get_expanded_df(coo_results, optim="adam")
coo_results.loc[coo_results["mean_squared_error"].idxmin()]

mean_squared_error                                     0.541137
date                                        2022-03-15_16-40-57
hostname                         holy7c24401.rc.fas.harvard.edu
seed                                                          5
d_lr                                      0.0020340456073678417
g_lr                                      0.0085935731462449955
gamma                                       0.97056402135198994
g_betas               [0.09474357764650297, 0.8598391909289105]
d_betas               [0.16471721585190355, 0.8167754867455241]
step_size                                                     4
d_n_hidden_units                                             50
g_n_hidden_units                                             20
d_n_hidden_layers                                             2
g_n_hidden_layers                                             2
Name: 142, dtype: object

In [36]:
# Read in the COO results
coo_results = pd.read_csv("classical_results/ray_tune_coo_SmoothL1Loss.csv", usecols=np.arange(1,22))
coo_results = get_expanded_df(coo_results, optim="adam")
coo_results.loc[coo_results["mean_squared_error"].idxmin()]

mean_squared_error                                   3.1224e-07
date                                        2022-03-15_17-20-24
hostname                         holy7c24401.rc.fas.harvard.edu
seed                                                          7
d_lr                                      0.0095599998204918078
g_lr                                       0.019509577339306788
gamma                                       0.99815883560751795
g_betas               [0.15350151781527333, 0.7878688096570859]
d_betas               [0.03476783931192041, 0.7636980059142577]
step_size                                                    15
d_n_hidden_units                                             30
g_n_hidden_units                                             40
d_n_hidden_layers                                             4
g_n_hidden_layers                                             4
Name: 82, dtype: object

# 7. WAV

## 7.1 L2

In [37]:
# Read in the WAV results
wav_results = pd.read_csv("classical_results/ray_tune_wav_MSELoss.csv", usecols=np.arange(1,22))
wav_results = get_expanded_df(wav_results, optim="adam")
wav_results.loc[wav_results["mean_squared_error"].idxmin()]

mean_squared_error                                  6.10793e-07
date                                        2022-03-15_17-48-35
hostname                         holy7c24404.rc.fas.harvard.edu
seed                                                          6
d_lr                                      0.0065757617830373514
g_lr                                       0.013816215479638938
gamma                                       0.99346846982349934
g_betas               [0.09065264061643237, 0.8968764927531144]
d_betas               [0.02996252331721059, 0.6707534659706094]
step_size                                                     7
d_n_hidden_units                                             40
g_n_hidden_units                                             40
d_n_hidden_layers                                             4
g_n_hidden_layers                                             2
Name: 136, dtype: object

## 7.2 L1

In [39]:
# Read in the WAV results
wav_results = pd.read_csv("classical_results/ray_tune_wav_L1Loss.csv", usecols=np.arange(1,22))
wav_results = get_expanded_df(wav_results, optim="adam")
wav_results.loc[wav_results["mean_squared_error"].idxmin()]

mean_squared_error                                    0.0004657
date                                        2022-03-15_18-03-56
hostname                         holy7c24404.rc.fas.harvard.edu
seed                                                          2
d_lr                                       0.001282459512594362
g_lr                                       0.018560757807083259
gamma                                       0.99384717026128189
g_betas                [0.11615063247434658, 0.937875470395432]
d_betas               [0.10238173755255711, 0.8280008663187798]
step_size                                                     8
d_n_hidden_units                                             40
g_n_hidden_units                                             50
d_n_hidden_layers                                             4
g_n_hidden_layers                                             5
Name: 136, dtype: object

## 7.3 Huber

In [40]:
# Read in the WAV results
wav_results = pd.read_csv("classical_results/ray_tune_wav_SmoothL1Loss.csv", usecols=np.arange(1,22))
wav_results = get_expanded_df(wav_results, optim="adam")
wav_results.loc[wav_results["mean_squared_error"].idxmin()]

mean_squared_error                                   2.81389e-07
date                                         2022-03-15_18-14-29
hostname                          holy7c24404.rc.fas.harvard.edu
seed                                                           0
d_lr                                        0.018971909791831287
g_lr                                       0.0046233575682293601
gamma                                        0.98570241091170008
g_betas               [0.009721396562610523, 0.7535189847954444]
d_betas                 [0.2336385689688982, 0.9598593678903538]
step_size                                                     15
d_n_hidden_units                                              40
g_n_hidden_units                                              50
d_n_hidden_layers                                              4
g_n_hidden_layers                                              5
Name: 13, dtype: object

# 8. BURV

## 8.1 L2

In [41]:
# Read in the BURV results
burv_results = pd.read_csv("classical_results/ray_tune_burv_MSELoss.csv", usecols=np.arange(1,22))
burv_results = get_expanded_df(burv_results, optim="adam")
burv_results.loc[burv_results["mean_squared_error"].idxmin()]

mean_squared_error                                  0.000138748
date                                        2022-03-15_18-27-33
hostname                         holy7c24102.rc.fas.harvard.edu
seed                                                          9
d_lr                                     0.00050654443719511501
g_lr                                       0.013391496636832775
gamma                                       0.99146948206449215
g_betas               [0.11978500799402188, 0.8293059214035681]
d_betas                [0.0137788693942495, 0.8913425138441484]
step_size                                                    14
d_n_hidden_units                                             40
g_n_hidden_units                                             50
d_n_hidden_layers                                             5
g_n_hidden_layers                                             4
Name: 53, dtype: object

In [42]:
# Read in the BURV results
burv_results = pd.read_csv("classical_results/ray_tune_burv_L1Loss.csv", usecols=np.arange(1,22))
burv_results = get_expanded_df(burv_results, optim="adam")
burv_results.loc[burv_results["mean_squared_error"].idxmin()]

mean_squared_error                                   0.00175234
date                                        2022-03-15_19-01-16
hostname                         holy7c24102.rc.fas.harvard.edu
seed                                                          4
d_lr                                       0.014855311508696095
g_lr                                      0.0044575994866025833
gamma                                       0.97906363989530221
g_betas               [0.20429427655131266, 0.9892358998841247]
d_betas                [0.2142203129518682, 0.7951664512592199]
step_size                                                    15
d_n_hidden_units                                             20
g_n_hidden_units                                             50
d_n_hidden_layers                                             3
g_n_hidden_layers                                             4
Name: 165, dtype: object

In [43]:
# Read in the BURV results
burv_results = pd.read_csv("classical_results/ray_tune_burv_SmoothL1Loss.csv", usecols=np.arange(1,22))
burv_results = get_expanded_df(burv_results, optim="adam")
burv_results.loc[burv_results["mean_squared_error"].idxmin()]

mean_squared_error                                 9.44136e-05
date                                       2022-03-15_19-25-04
hostname                        holy7c24102.rc.fas.harvard.edu
seed                                                         2
d_lr                                    0.00056124536386726242
g_lr                                       0.01526412112125624
gamma                                      0.98753466629554609
g_betas               [0.2368080488188987, 0.6900329210360248]
d_betas                [0.1475522620084919, 0.953530294043536]
step_size                                                    9
d_n_hidden_units                                            20
g_n_hidden_units                                            40
d_n_hidden_layers                                            4
g_n_hidden_layers                                            5
Name: 106, dtype: object

# 9. EINS

## 9.1 L2

In [3]:
# Read in the EINS results
eins_results = pd.read_csv("classical_results/ray_tune_eins_MSELoss.csv", usecols=np.arange(1,22))
eins_results = get_expanded_df(eins_results, optim="adam")
eins_results.loc[eins_results["mean_squared_error"].idxmin()]

mean_squared_error                                    0.0243689
date                                        2022-03-15_22-32-20
hostname                         holy7c24503.rc.fas.harvard.edu
seed                                                          5
d_lr                                     0.00054592204924509813
g_lr                                       0.015651765550030356
gamma                                        0.9084080156243779
g_betas                [0.3278484531625797, 0.9569560300463275]
d_betas               [0.01810586599566555, 0.7760976169368717]
step_size                                                     7
d_n_hidden_units                                             20
g_n_hidden_units                                             40
d_n_hidden_layers                                             3
g_n_hidden_layers                                             4
Name: 38, dtype: object

## 9.2 L1

In [4]:
# Read in the EINS results
eins_results = pd.read_csv("classical_results/ray_tune_eins_L1Loss.csv", usecols=np.arange(1,22))
eins_results = get_expanded_df(eins_results, optim="adam")
eins_results.loc[eins_results["mean_squared_error"].idxmin()]

mean_squared_error                                   0.0578004
date                                       2022-03-15_23-32-46
hostname                        holy7c24503.rc.fas.harvard.edu
seed                                                         2
d_lr                                     0.0072075600407168197
g_lr                                     0.0044761573135886313
gamma                                      0.90038443210143093
g_betas               [0.2651217922263499, 0.8713242023842087]
d_betas               [0.3140320415033071, 0.9871301484344451]
step_size                                                   20
d_n_hidden_units                                            30
g_n_hidden_units                                            20
d_n_hidden_layers                                            2
g_n_hidden_layers                                            5
Name: 67, dtype: object

## 9.3 Huber

In [5]:
# Read in the EINS results
eins_results = pd.read_csv("classical_results/ray_tune_eins_SmoothL1Loss.csv", usecols=np.arange(1,22))
eins_results = get_expanded_df(eins_results, optim="adam")
eins_results.loc[eins_results["mean_squared_error"].idxmin()]

mean_squared_error                                    0.0119287
date                                        2022-03-16_01-04-01
hostname                         holy7c24503.rc.fas.harvard.edu
seed                                                          2
d_lr                                      0.0052266747729013746
g_lr                                      0.0032874875413043654
gamma                                       0.91962328976829422
g_betas               [0.01317438914504253, 0.8572572799344018]
d_betas                [0.1366958427855389, 0.6939722021645833]
step_size                                                     6
d_n_hidden_units                                             20
g_n_hidden_units                                             40
d_n_hidden_layers                                             5
g_n_hidden_layers                                             4
Name: 195, dtype: object

# 10. HEA

## 10.1 L2

In [44]:
# Read in the HEA results
hea_results = pd.read_csv("classical_results/ray_tune_hea_MSELoss.csv", usecols=np.arange(1,22))
hea_results = get_expanded_df(hea_results, optim="adam")
hea_results.loc[hea_results["mean_squared_error"].idxmin()]

mean_squared_error                                   4.06054e-08
date                                         2022-03-15_22-24-44
hostname                          holy7c24302.rc.fas.harvard.edu
seed                                                           0
d_lr                                        0.011766998769263511
g_lr                                        0.011452866991177446
gamma                                         0.9927042396991933
g_betas                [0.09787518253758568, 0.6744606058542391]
d_betas               [0.043345073526828405, 0.9544479706202283]
step_size                                                      7
d_n_hidden_units                                              30
g_n_hidden_units                                              50
d_n_hidden_layers                                              5
g_n_hidden_layers                                              3
Name: 18, dtype: object

## 10.2 L1

In [45]:
# Read in the HEA results
hea_results = pd.read_csv("classical_results/ray_tune_hea_L1Loss.csv", usecols=np.arange(1,22))
hea_results = get_expanded_df(hea_results, optim="adam")
hea_results.loc[hea_results["mean_squared_error"].idxmin()]

mean_squared_error                                  0.000169207
date                                        2022-03-15_22-37-44
hostname                         holy7c24302.rc.fas.harvard.edu
seed                                                          7
d_lr                                        0.01494633283066788
g_lr                                      0.0075860491856194869
gamma                                       0.97092008999121848
g_betas                [0.0366220046870521, 0.9157607003011348]
d_betas               [0.11175353790149153, 0.8728324033109129]
step_size                                                    20
d_n_hidden_units                                             40
g_n_hidden_units                                             50
d_n_hidden_layers                                             2
g_n_hidden_layers                                             2
Name: 190, dtype: object

## 10.3 Huber

In [46]:
# Read in the HEA results
hea_results = pd.read_csv("classical_results/ray_tune_hea_SmoothL1Loss.csv", usecols=np.arange(1,22))
hea_results = get_expanded_df(hea_results, optim="adam")
hea_results.loc[hea_results["mean_squared_error"].idxmin()]

mean_squared_error                                  2.30842e-08
date                                        2022-03-15_22-45-04
hostname                         holy7c24302.rc.fas.harvard.edu
seed                                                          7
d_lr                                       0.009083455861739885
g_lr                                      0.0070004156190028007
gamma                                       0.96863557435513437
g_betas               [0.11940827221249413, 0.7180508670934431]
d_betas                 [0.2435135680932003, 0.965176701754245]
step_size                                                    10
d_n_hidden_units                                             20
g_n_hidden_units                                             50
d_n_hidden_layers                                             4
g_n_hidden_layers                                             4
Name: 73, dtype: object

# 11. ACA

## 11.1 L2

In [6]:
# Read in the ACA results with lambda tuning with no reparameterization
aca_results = pd.read_csv("classical_results/ray_tune_aca_MSELoss.csv", usecols=np.arange(1,22))
aca_results = get_expanded_df(aca_results, optim="adam", lam=True)
aca_results.loc[aca_results["mean_squared_error"].idxmin()]

mean_squared_error                                     0.0105075
date                                         2022-03-16_10-08-56
hostname                          holy7c24102.rc.fas.harvard.edu
seed                                                           6
d_lr                                       0.0092630235357480696
g_lr                                       0.0048500037296834641
gamma                                        0.99683849753079745
g_betas                [0.27218549347819215, 0.8707436271946659]
d_betas               [0.045357717680438045, 0.6812314685504909]
step_size                                                      5
d_n_hidden_units                                              20
g_n_hidden_units                                              20
d_n_hidden_layers                                              2
g_n_hidden_layers                                              5
lam                                           1.2329491581573497
Name: 116, dtype: object

## 11.2 L1

In [7]:
# Read in the ACA results with lambda tuning with no reparameterization
aca_results = pd.read_csv("classical_results/ray_tune_aca_L1Loss.csv", usecols=np.arange(1,22))
aca_results = get_expanded_df(aca_results, optim="adam", lam=True)
aca_results.loc[aca_results["mean_squared_error"].idxmin()]

mean_squared_error                                   0.0504081
date                                       2022-03-16_13-22-38
hostname                        holy7c24102.rc.fas.harvard.edu
seed                                                         2
d_lr                                      0.016701338390688859
g_lr                                     0.0038304582878111353
gamma                                      0.99142043338589414
g_betas               [0.29603548221226555, 0.897728618892474]
d_betas               [0.2956084580127263, 0.7863734810882368]
step_size                                                    7
d_n_hidden_units                                            30
g_n_hidden_units                                            40
d_n_hidden_layers                                            4
g_n_hidden_layers                                            5
lam                                         1.0518768255769018
Name: 3, dtype: object

## 11.3 Huber

In [8]:
# Read in the ACA results with lambda tuning with no reparameterization
aca_results = pd.read_csv("classical_results/ray_tune_aca_SmoothL1Loss.csv", usecols=np.arange(1,22))
aca_results = get_expanded_df(aca_results, optim="adam", lam=True)
aca_results.loc[aca_results["mean_squared_error"].idxmin()]

mean_squared_error                                    0.0031253
date                                        2022-03-16_17-28-34
hostname                         holy7c24402.rc.fas.harvard.edu
seed                                                          3
d_lr                                       0.011790447592709467
g_lr                                      0.0055679181189701903
gamma                                       0.99395341175393637
g_betas               [0.15583328179997716, 0.8582490052093621]
d_betas               [0.09457624953523017, 0.7619888601022959]
step_size                                                    20
d_n_hidden_units                                             20
g_n_hidden_units                                             20
d_n_hidden_layers                                             2
g_n_hidden_layers                                             2
lam                                          1.2876835878247825
Name: 114, dtype: object

# 12. RAYS

## 12.1 Regular GAN

In [38]:
# Read in the RAYS results
rays_results = pd.read_csv("gan_results/ray_tune_rays_fixed.csv", usecols=np.arange(1,22))
rays_results = get_expanded_df(rays_results, optim="adam")
rays_results.loc[rays_results.groupby("seed")["mean_squared_error"].idxmin()]

,mean_squared_error,date,hostname,seed,d_lr,g_lr,gamma,g_betas,d_betas,step_size,d_n_hidden_units,g_n_hidden_units,d_n_hidden_layers,g_n_hidden_layers
487,7.754963e-08,2022-02-15_09-12-30,holy7c24303.rc.fas.harvard.edu,2,0.0041272976272267361,0.0093581768678081979,0.97893579812771658,"[0.21932425228717567, 0.7154610394492373]","[0.15260135969704555, 0.7677411091156993]",13,20,50,2,2
